# Data

This notebook was created for machine learning research on FeTA2.1 dataset. ["The dataset facilitates the development of novel machine-learning and deep-learning based multi-class segmentation methods for the quantification of brain development on fetal MRI. The ultimate goal is to capture pathological developmental trajectories by the automated quantification of the prenatal development, for which automated approaches free of observer bias are indispensable."](http://neuroimaging.ch/feta) Some brief information about the labels can be found below. For more information, please follow the links in the notes.

| dHCP label | Name | Notes |
| :- | -: | :-: |
| Label 1 | Intracranial space and extra-axial CSF spaces | Cerebrospinal fluid (CSF) is a clear, colorless body fluid found within the tissue that surrounds the brain and spinal cord of all vertebrates.[[1]](https://en.wikipedia.org/wiki/Cerebrospinal_fluid) 
| Label 2 | Gray Matter | Grey matter (or gray matter) is a major component of the central nervous system, consisting of neuronal cell bodies, neuropil (dendrites and unmyelinated axons), glial cells (astrocytes and oligodendrocytes), synapses, and capillaries.[[2]](https://en.wikipedia.org/wiki/Grey_matter) 
| Label 3 | White Matter | White matter refers to areas of the central nervous system (CNS) that are mainly made up of myelinated axons, also called tracts. [[3]](https://en.wikipedia.org/wiki/White_matter)
| Label 4 | Ventricles | The ventricles are structures that produce cerebrospinal fluid, and transport it around the cranial cavity. [[4]](https://teachmeanatomy.info/neuroanatomy/vessels/ventricles/)
| Label 5 | Cerebellum | The cerebellum (which is Latin for “little brain”) is a major structure of the hindbrain that is located near the brainstem. This part of the brain is responsible for coordinating voluntary movements. It is also responsible for a number of functions including motor skills such as balance, coordination, and posture. [[5]](https://www.verywellmind.com/what-is-the-cerebellum-2794964)
| Label 6 | Deep gray matter | Grey matter is classed as either superficial or deep. The superficial grey matter, also called brain cortex, is on the outside of the brain. The deep grey matter is made up of neurons from which originate deep nerve fibres. These neurons tend to form clumps of basal nuclei.[[6]](https://www.msif.org/news/2014/12/16/role-deep-grey-matter-brain-plays-ms/)
| Label 7 | Brainstem and spinal cord| The brainstem (or brain stem) is the posterior stalk-like part of the brain that connects the cerebrum with the spinal cord. [[7]](https://en.wikipedia.org/wiki/Brainstem)

<br><br>

![Structure of the brain](notebook_images/brain2.jpg "Structure of the brain 1.")
<br><p><center>Structure of the brain 1.<a href="https://www.kenhub.com/en/library/learning-strategies/parts-of-the-brain-learn-with-diagrams-and-quizzes">[8]</a></center></p>
<br><br>

![Structure of the brain](notebook_images/brain3.jpg "Structure of the brain 2.")
<br><p><center>Structure of the brain 2.<a href="https://www.researchgate.net/publication/224283818_Patientoriented_simulation_based_on_Monte_Carlo_algorithm_by_using_MRI_data">[9]</a></center></p>
<br><br>

In [1]:
import glob
import json
import os

import cv2
import matplotlib.colorbar as colorbar
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from ipywidgets import fixed, interactive

In [2]:
def get_subjects(folder):
    subjects = glob.glob(os.path.join(folder, 's*'))
    subjects = [os.path.basename(x) for x in subjects]
    subjects.sort()

    return subjects

In [3]:
def get_nii_paths(subject):
    suffix_image = "_rec-mial_T2w.nii.gz"
    suffix_mask = "_rec-mial_dseg.nii.gz"
    
    path_sub = os.path.join(folder, 
                              subject, 
                              "anat", 
                              subject)
    
    path_image = path_sub+suffix_image
    
    path_mask = path_sub+suffix_mask
    
    return (path_image, path_mask)

In [4]:
class COLORS():
    def __init__(self):
        self.colors = {0: (0, 0, 0),          #Black for background.
                  1: (128,128,0),             #Olive for CSF.
                  2: (211,211,211),           #Light gray for gray matter.
                  3: (255, 255, 255),         #White for white matter.
                  4: (0,0,128),               #Navy for ventricles.
                  5: (255,69,0),              #Orange red for cerebellum. 
                  6: (105,105,105),           #Dim gray for deep gray matter.
                  7: (255,255,224)            #Light yellow for brainstem and spinal cord.
         }

In [5]:
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    
    return result

In [6]:
def convert_to_3D(mask):
    zeros = np.zeros((256, 256, 2))
    mask_rgb = np.stack((zeros[:, :, 0], zeros[:, :, 1], mask), axis=2).astype(int)
    
    return mask_rgb

In [7]:
def change_colors(mask, colors):
    mask = mask.astype(int)
    labels = set(mask[:,:,2].flatten())
        
    for label in labels:
        mask[mask[:, :, 2] == label] = colors[label]
        
    return mask

In [8]:
def show_image_mask(image, mask):
    image = rotate_image(image, 90)
        
    mask = rotate_image(mask, 90)
    mask_3d = convert_to_3D(mask)
    colors = COLORS()
    mask_rgb = change_colors(mask_3d, colors.colors)
    
    return image, mask_rgb

In [9]:
def draw_layout(index, sub_name, orientation):
    (path_image, path_mask) = get_nii_paths(sub_name)
    
    image = nib.load(path_image).get_fdata()
    mask = nib.load(path_mask).get_fdata()
    
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 15))
    ax1.set_title('Image')
    ax2.set_title('Mask')
    

    if orientation == "coronal":
        image, mask = show_image_mask(image[:, index, :],        
                                mask[:, index, :])
        
        ax1.imshow(image, cmap='gray')
        ax2.imshow(mask)
        
    elif orientation == "axial":
        image, mask = show_image_mask(image[:, :, index],
                                      mask[:, :, index])
                
        im = ax1.imshow(image, cmap='gray')
        im = ax2.imshow(mask)

    else:
        image, mask = show_image_mask(image[index, :, :],
                                      mask[index, :, :])
        
        mask = cv2.flip(mask, 1)
        
        ax1.imshow(image, cmap='gray')
        ax2.imshow(mask)
    

def show_data(subject):
    interactive_plot = interactive(draw_layout, index=(0, 255),
                                   sub_name = fixed(subject),
                                   orientation=["axial", "coronal", "sagittal"])
    
    output = interactive_plot.children[-1]
    display(interactive_plot)

In [10]:
folder = 'feta_2.1/'

subjects = get_subjects(folder)

In [11]:
show_data(subjects[0])

interactive(children=(IntSlider(value=127, description='index', max=255), Dropdown(description='orientation', …

In [ ]:
def draw_colorbar():
    cb_colors = ["#000000", "#808000", "#D3D3D3", "#FFFFFF", "#000080", "#FF4500", "#696969", "#FFFFE0"]
    cb_labels = ["background", "CSF", "gray matter", "white matter", 
                        "ventricles", "cerebellum", "deep gray matter", "brain stem spinal cord"]
    cmap_ = clr.ListedColormap(cb_colors)
    
    
    fig = plt.figure()
    ax = fig.add_axes([0.05, 0.80, 0.9, 0.1])
    
    cb = colorbar.ColorbarBase(ax, orientation='horizontal',
                               cmap=cmap_, norm=plt.Normalize(-0.5, len(cb_colors) - 0.5))
    
    
    cb.set_ticks(range(len(cb_colors)))
    cb.ax.set_xticklabels(cb_labels, rotation=45)

    return cb

In [ ]:


cb_colors = ["#000000", "#808000", "#D3D3D3", "#FFFFFF", "#000080", "#FF4500", "#696969", "#FFFFE0"]
cb_labels = ["background", "CSF", "gray matter", "white matter", 
                    "ventricles", "cerebellum", "deep gray matter", "spinal cord"]
cmap_ = clr.ListedColormap(cb_colors)


fig = plt.figure()
ax = fig.add_axes([0.05, 0.80, 0.9, 0.1])

cb = colorbar.ColorbarBase(ax, orientation='horizontal',
                           cmap=cmap_, norm=plt.Normalize(-0.5, len(cb_colors) - 0.5))


cb.set_ticks(range(len(cb_colors)))
cb.ax.set_xticklabels(cb_labels, rotation=45)
plt.show()